In [4]:
import numpy as np
import matplotlib.pyplot as plt

%run -i sedfuncs.py

In [3]:
# Calculate wavenumber

T = 10. # wave period (s)
h = 5.  # water depth (m)

kh = qkhfs(2.*np.pi/T, h)
print(kh, kh/h)

0.4641801957195109 0.09283603914390218


In [6]:
# Calculate wave-orbital velocity from Hs, T, h
Hs = 1.
ub = ub_func(T, kh/h, Hs, h)
print(ub)

0.6530974940757303
